1. PARSE XML FILES
2. TURN INTO TREE
3. IF FAILS, PARSE AS MANY OF THE ENTRIES AS POSSIBLE

In [4]:
import traceback
import xml.etree.ElementTree as ET
from pathlib import Path
import re
import shutil
import os

import git

repo = git.Repo('.', search_parent_directories=True)
os.chdir(repo.working_tree_dir)

srcpaths=(
    "/run/media/benjis/basilisk/Files/biggie/oss-fuzz/fuzz_10m_trace_3h/trace_run_3_success_3h/traces-10m/logs-xmls",
)

xmls = []
for d in srcpaths:
    l = list(Path(d).glob("*.xml"))
    l = [p for p in l if ".repair." not in p.name]
    xmls += l
print(len(xmls), "XML files found,", len(set([f.name for f in xmls])), "unique. duplicates =", [f for f in xmls if [g.name for g in xmls].count(f.name) > 1])

178 XML files found, 178 unique. duplicates = []


In [5]:
dstdir=Path("postprocessed_xmls")

In [19]:
if dstdir.exists():
    shutil.rmtree(dstdir)
dstdir.mkdir(exist_ok=True)

In [20]:
import tqdm as tqdm

successes_1 = []
failed_1 = []
for fpath in tqdm.tqdm(xmls):
    try:
        it = ET.iterparse(fpath)
        for tag in it:
            pass
        successes_1.append(fpath)
        shutil.copyfile(fpath, dstdir/fpath.name)
    except ET.ParseError as ex:
        failed_1.append(fpath)

print("ROUND 1:", len(failed_1), "files failed parsing")
(dstdir/"1_repair_success_1.txt").write_text("\n".join(map(str, successes_1)))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 178/178 [1:20:47<00:00, 27.24s/it]


ROUND 1: 150 files failed parsing


4367

In [10]:
import tqdm as tqdm

fuzzer_start = re.compile(r'''<call[^>]*method="[^("]+.fuzzerTestOneInput\([^)]+\)"[^>]*''')

from xml.dom import minidom
from xml.parsers.expat import ExpatError

def recover_functions_simple(fpath, dstdir):
    repair_path = dstdir/(str(fpath.name) + ".repair.xml")
    if repair_path.exists():
        return repair_path
    
#     with open(fpath) as f:
#         num_lines = sum(1 for line in tqdm.tqdm(f, desc=f"count lines ({fpath.name})"))
    last_fuzzer_start = None
    with open(fpath) as f:
        i = 0
        it = iter(f)
        failed_functions = 0
        all_functions = 0
        with tqdm.tqdm(it, total=None, desc=f"chop final fuzzer target call ({fpath.name})") as pbar:
            it = pbar
            fuzz_functions = []
            current_fuzz_function = None
            for i, line in enumerate(it):
                m = fuzzer_start.search(line)
                if m is not None:
#                     print("MATCH", m, m.start(), m.span())
                    j = m.start()
                    last_fuzzer_start = (i, j)
#     print("LFS", last_fuzzer_start)
    if last_fuzzer_start is None:
        print("ERROR: last_fuzzer_start is None", fpath)
        return None
    
    with open(fpath) as f, open(repair_path, "w") as outf:
        it = iter(f)
        with tqdm.tqdm(it, total=last_fuzzer_start[0], desc=f"write file ({fpath.name})") as pbar:
            it = pbar
            for i, line in enumerate(it):
#                 print(i)
                if i == last_fuzzer_start[0]:
#                     print("LINE:")
#                     print(line)
                    line_chopped = line[:last_fuzzer_start[1]]
#                     print("CHOPPED:")
#                     print(line_chopped)
                    outf.write(line_chopped)
                    break
                else:
                    outf.write(line)
            outf.write("</trace>")
        
    return repair_path

xml_file = Path("/run/media/benjis/basilisk/Files/biggie/oss-fuzz/fuzz_10m_trace_3h/trace_run_3_success_3h/traces-10m/logs-xmls")/"trace-apache-commons-cli-ParserFuzzer.xml"
recover_functions_simple(xml_file, Path("postprocessed_xmls_debug"))

PosixPath('postprocessed_xmls_debug/trace-apache-commons-cli-ParserFuzzer.xml.repair.xml')

In [ ]:
fpath = Path("/run/media/benjis/basilisk/Files/biggie/oss-fuzz/fuzz_10m_trace_3h/trace_run_3_success_3h/traces-10m/logs-xmls")/"trace-apache-commons-cli-ParserFuzzer.xml"
repair_path = recover_functions_simple(fpath, Path("postprocessed_xmls_debug"))
print(fpath, "->", repair_path)
it = ET.iterparse(repair_path)
for tag in tqdm.tqdm(it, desc=f"check XML parse ({fpath.name})"):
    pass

/run/media/benjis/basilisk/Files/biggie/oss-fuzz/fuzz_10m_trace_3h/trace_run_3_success_3h/traces-10m/logs-xmls/trace-apache-commons-cli-ParserFuzzer.xml -> postprocessed_xmls_debug/trace-apache-commons-cli-ParserFuzzer.xml.repair.xml


check XML parse (trace-apache-commons-cli-ParserFuzzer.xml): 1898417it [00:17, 155880.30it/s]

In [8]:
successes_1 = list(map(Path, (dstdir/"1_repair_success_1.txt").read_text().split("\n")))
failed_1 = [fpath for fpath in xmls if fpath.name not in [s.name for s in successes_1]]
len(successes_1), len(failed_1)

(28, 150)

In [9]:
import tqdm
successes_2 = []
failed_2 = []
for fpath in tqdm.tqdm(failed_1, position=1, desc="round 2"):
    repair_path = recover_functions_simple(fpath, dstdir)
    try:
        it = ET.iterparse(repair_path)
        for tag in tqdm.tqdm(it, desc=f"check XML parse ({fpath.name})"):
            pass
        successes_2.append(repair_path)
        # shutil.copyfile(repair_path, dstdir/repair_path.name)
    except ET.ParseError as ex:
        print("exception", type(ex), ex, "parsing", fpath, repair_path)
        failed_2.append(repair_path)

print("ROUND 2:", len(failed_2), "files failed parsing")
(dstdir/"1_repair_success_2.txt").write_text("\n".join(map(str, successes_2)))


chop final fuzzer target call (trace-jackson-dataformat-xml-XmlFuzzer.xml): 40372it [00:00, 248527.45it/s]                                             | 0/150 [00:00<?, ?it/s]
write file (trace-jackson-dataformat-xml-XmlFuzzer.xml): 100%|█████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 7219.11it/s]
check XML parse (trace-jackson-dataformat-xml-XmlFuzzer.xml): 1it [00:00, 4048.56it/s]

chop final fuzzer target call (trace-apache-commons-bcel-BcelFuzzer.xml): 83662it [00:00, 407728.59it/s]                                       | 1/150 [00:00<01:10,  2.12it/s]
write file (trace-apache-commons-bcel-BcelFuzzer.xml): 100%|█████████████████████████████████████████████████████████████████████████| 83212/83212 [00:00<00:00, 805007.01it/s]
check XML parse (trace-apache-commons-bcel-BcelFuzzer.xml): 81623it [00:00, 184098.44it/s]

chop final fuzzer target call (trace-apache-commons-beanutils-BeanutilsFuzzer.xml): 5651425it [00:13, 412016.00it/s

exception <class 'xml.etree.ElementTree.ParseError'> mismatched tag: line 3751875, column 2 parsing /run/media/benjis/basilisk/Files/biggie/oss-fuzz/fuzz_10m_trace_3h/trace_run_3_success_3h/traces-10m/logs-xmls/trace-apache-commons-cli-ParserFuzzer.xml postprocessed_xmls/trace-apache-commons-cli-ParserFuzzer.xml.repair.xml


chop final fuzzer target call (trace-apache-commons-codec-BinaryEncodingFuzzer.xml): 3744975it [00:10, 346069.65it/s]
write file (trace-apache-commons-codec-BinaryEncodingFuzzer.xml): 100%|█████████████████████████████████████████████████████████| 3710705/3710705 [00:03<00:00, 1064691.17it/s]
check XML parse (trace-apache-commons-codec-BinaryEncodingFuzzer.xml): 3796845it [00:27, 138965.04it/s]

chop final fuzzer target call (trace-apache-commons-collections-CollectionsBidiMapFuzzer.xml): 14it [00:00, 362.07it/s]                      | 5/150 [02:19<1:24:31, 34.98s/it]
write file (trace-apache-commons-collections-CollectionsBidiMapFuzzer.xml): 100%|██████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 3830.41it/s]
check XML parse (trace-apache-commons-collections-CollectionsBidiMapFuzzer.xml): 1it [00:01,  1.05s/it]

chop final fuzzer target call (trace-apache-commons-CompressSevenZFuzzer.xml): 881221it [00:02, 407108.50it/s]                                 |

exception <class 'xml.etree.ElementTree.ParseError'> mismatched tag: line 4666420, column 2 parsing /run/media/benjis/basilisk/Files/biggie/oss-fuzz/fuzz_10m_trace_3h/trace_run_3_success_3h/traces-10m/logs-xmls/trace-apache-commons-CompressTarFuzzer.xml postprocessed_xmls/trace-apache-commons-CompressTarFuzzer.xml.repair.xml


chop final fuzzer target call (trace-apache-commons-CompressZipFuzzer.xml): 4267637it [00:10, 400933.18it/s]
write file (trace-apache-commons-CompressZipFuzzer.xml): 100%|██████████████████████████████████████████████████████████████████| 4250755/4250755 [00:03<00:00, 1136968.28it/s]
check XML parse (trace-apache-commons-CompressZipFuzzer.xml): 4260591it [00:32, 130518.39it/s]

chop final fuzzer target call (trace-apache-commons-configuration-INIConfigurationWriteFuzzer.xml): 42805it [00:00, 267027.36it/s]           | 9/150 [04:08<1:21:51, 34.83s/it]
write file (trace-apache-commons-configuration-INIConfigurationWriteFuzzer.xml): 100%|███████████████████████████████████████████████| 20055/20055 [00:00<00:00, 440420.37it/s]
check XML parse (trace-apache-commons-configuration-INIConfigurationWriteFuzzer.xml): 18380it [00:01, 15225.79it/s]

round 2:   7%|████████▌                                                                                                                       | 10/150

...

exception <class 'xml.etree.ElementTree.ParseError'> mismatched tag: line 20, column 2 parsing /run/media/benjis/basilisk/Files/biggie/oss-fuzz/fuzz_10m_trace_3h/trace_run_3_success_3h/traces-10m/logs-xmls/trace-h2database-SqlPreparedStatementFuzzer.xml postprocessed_xmls/trace-h2database-SqlPreparedStatementFuzzer.xml.repair.xml


chop final fuzzer target call (trace-h2database-SqlStatementFuzzer.xml): 141719it [00:00, 386279.92it/s]
write file (trace-h2database-SqlStatementFuzzer.xml): 100%|████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 106968.83it/s]
check XML parse (trace-h2database-SqlStatementFuzzer.xml): 3it [00:00, 12774.53it/s]

round 2:  37%|███████████████████████████████████████████████▊                                                                                | 56/150 [22:36<18:00, 11.50s/it]

exception <class 'xml.etree.ElementTree.ParseError'> mismatched tag: line 20, column 2 parsing /run/media/benjis/basilisk/Files/biggie/oss-fuzz/fuzz_10m_trace_3h/trace_run_3_success_3h/traces-10m/logs-xmls/trace-h2database-SqlStatementFuzzer.xml postprocessed_xmls/trace-h2database-SqlStatementFuzzer.xml.repair.xml


chop final fuzzer target call (trace-hamcrest-HamcrestFuzzer.xml): 547it [00:00, 30877.83it/s]
write file (trace-hamcrest-HamcrestFuzzer.xml): 100%|█████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 16644.06it/s]
check XML parse (trace-hamcrest-HamcrestFuzzer.xml): 1it [00:00, 4723.32it/s]
chop final fuzzer target call (trace-hsqldb-ConnectionOptionsFuzzer.xml): 5009it [00:00, 191181.06it/s]
round 2:  38%|████████████████████████████████████████████████▋                                                                               | 57/150 [22:36<36:52, 23.79s/it]


UnboundLocalError: cannot access local variable 'last_fuzzer_start' where it is not associated with a value

In [ ]:
all_files = successes_1 + successes_2

In [ ]:
with open(dstdir/"1_repair_success_all.txt", "w") as f:
    f.write("\n".join(map(str, all_files)))